In [1]:
import syft as sy
from syft.lib import python
sy.VERBOSE = False
from syft.core.common.uid import UID

adding, path <class 'syft.ast.klass.Class'>
adding, path <class 'syft.ast.klass.Class'>
adding, path <class 'syft.ast.klass.Class'>
what is our attr_path and name models.MNIST.__call__
what is our attr_path and name models.MNIST.parameters
what is our attr_path and name models.MNIST.to
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> zeros_like ['torch', 'zeros_like'] <syft.ast.function.Function object at 0x1336c8460> <class 'syft.ast.function.Function'>
adding, path <class 'syft.ast.function.Function'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'T'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'T'] Module:
 <class 'syft.ast.module.Mod

We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'cosh'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'cosh'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', 'cosh'] <Class:Parameter> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'cosh_'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'cosh_'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter

We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'mm'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', 'mm'] <Class:Parameter> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'mul'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'mul'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', 'mul'] <Class:Parameter> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'mu

 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', 'unique_consecutive'] <Class:Parameter> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'unsqueeze'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'unsqueeze'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', 'unsqueeze'] <Class:Parameter> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'unsqueeze_'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'

In [2]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_root_client()
alice.root_verify_key = alice_client.verify_key  # inject 📡🔑 as 📍🗝

adding, path <class 'syft.ast.klass.Class'>
adding, path <class 'syft.ast.klass.Class'>
adding, path <class 'syft.ast.klass.Class'>
what is our attr_path and name models.MNIST.__call__
what is our attr_path and name models.MNIST.parameters
what is our attr_path and name models.MNIST.to
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> zeros_like ['torch', 'zeros_like'] <syft.ast.function.Function object at 0x133cfae20> <class 'syft.ast.function.Function'>
adding, path <class 'syft.ast.function.Function'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'T'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'T'] Module:
 <class 'syft.ast.module.Mod

We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'div_'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'div_'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', 'div_'] <Class:Parameter> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'dot'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'dot'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', '

adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'neg_'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', 'neg_'] <Class:Parameter> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Tensor ['torch', 'Tensor', 'new_tensor'] <Class:Tensor> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> nn ['torch', 'nn', 'Parameter', 'new_tensor'] Module:
	.Parameter -> <Class:Parameter>
 <class 'syft.ast.module.Module'>
We need to add methods <class 'syft.ast.module.Module'> Parameter ['torch', 'nn', 'Parameter', 'new_tensor'] <Class:Parameter> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add m

We need to add methods <class 'syft.ast.module.Module'> Generator ['torch', 'Generator'] <Class:Generator> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Generator ['torch', 'Generator', 'get_state'] <Class:Generator> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> Generator ['torch', 'Generator', 'set_state'] <Class:Generator> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> device ['torch', 'device'] <Class:device> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> device ['torch', 'device', 'index'] <Class:device> <class 'abc.SubclassableClass'>
adding, path <class 'abc.SubclassableClass'>
We need to add methods <class 'syft.ast.module.Module'> device 

In [3]:
# original MNIST imports
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torch.optim.lr_scheduler import StepLR

# get imports from remote client to match
torch = alice_client.torch
torchvision = alice_client.torchvision
transforms = torchvision.transforms
datasets = torchvision.datasets
nn = torch.nn
F = torch.nn.functional
optim = torch.optim
StepLR = torch.optim.lr_scheduler.StepLR
Net = alice_client.models.MNIST

In [4]:
print(alice_client.lib_ast(path="syft.lib.python.String", return_callable=True))

we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'String'], 'index': 4} True syft.lib.python.String
<Class:String>


In [5]:
# helper to create and send a primitive
def make(thing):
    prim = python.primitive_factory.PrimitiveFactory.generate_primitive(value=thing, recurse=True)
    ptr = prim.send(alice_client)
    return ptr

In [6]:
device = torch.device(make("cpu"))
print(type(device))

we are calling <class 'abc.SubclassableClass'> (<syft.proxy.syft.lib.python.StringPointer object at 0x104c28910>,) {} False torch.device
call ast with  torch.device True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device'], 'index': 2} True torch.device
this is our return tensor <class 'syft.proxy.torch.devicePointer'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'String'], 'index': 4} True syft.lib.python.String
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device'], 'index': 2} False torch.device
we are here for <class 'abc.SubclassableClass'> <class 'torch.device'>
<class 'syft.proxy.torch.devicePointer'>


In [7]:
print(alice.store.peek)

{
  > <UID:d78bf8c7-d063-4add-ba79-46f85554ba69> <UID:🚴🙕> => 🗂️ (StorableObject) ('cpu' can_read: ['🚻', '🛺'])

  > <UID:b6763f66-dd2b-4a57-961f-18a47c650213> <UID:🙈🚇> => 🗂️ (StorableObject) (device(type='cpu') can_read: ['🚻', '🛺'])
}


In [8]:
print(device.id_at_location)

<UID:b6763f66-dd2b-4a57-961f-18a47c650213>


In [9]:
a = device.type
print(a)

call property type <bound method Class.create_pointer_class.<locals>.get_run_class_method.<locals>.run_class_method of <syft.proxy.torch.devicePointer object at 0x133da33a0>>
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device', 'type'], 'index': 2} True torch.device
we are calling <class 'syft.ast.method.Method'> () {'path': ['torch', 'device', 'type'], 'index': 3} True syft.lib.python.String
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'String'], 'index': 4} True syft.lib.python.String
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device'], 'index': 2} True torch.device
executing a  <class 'syft.core.node.common.action.run_class_method_action.RunClassMethodAction'>
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device', 'type'], 'index': 2} False torch.device
we are calling <class 'syft.ast.method.Method'> () {'path': ['torch', 'device', 'type'], 'index': 3} False syft.lib.p

In [10]:
a.get()

'cpu'

In [11]:
print(alice.store.peek)

{
  > <UID:d78bf8c7-d063-4add-ba79-46f85554ba69> <UID:🚴🙕> => 🗂️ (StorableObject) ('cpu' can_read: ['🚻', '🛺'])

  > <UID:b6763f66-dd2b-4a57-961f-18a47c650213> <UID:🙈🚇> => 🗂️ (StorableObject) (device(type='cpu') can_read: ['🚻', '🛺'])
}


In [12]:
print(dtype, type(dtype))

NameError: name 'dtype' is not defined

In [13]:
#alice_client.torch.device.attrs["type"].return_type_name

return_type_name = alice_client.lib_ast("torch.device.type", return_callable=True).return_type_name
print(return_type_name)

a = alice_client.lib_ast(return_type_name, return_callable=True)
print(a.pointer_type)

we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device', 'type'], 'index': 2} True torch.device
we are calling <class 'syft.ast.method.Method'> () {'path': ['torch', 'device', 'type'], 'index': 3} True syft.lib.python.String
syft.lib.python.String
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'String'], 'index': 4} True syft.lib.python.String
<class 'syft.proxy.syft.lib.python.StringPointer'>


In [14]:
attr = object.__getattribute__(device, "type")

In [15]:
print(attr)
#attr = object.__getattribute__(__self, name)

<bound method Class.create_pointer_class.<locals>.get_run_class_method.<locals>.run_class_method of <syft.proxy.torch.devicePointer object at 0x133da33a0>>


In [16]:
device.type

#method.__get__(data)

call property type <bound method Class.create_pointer_class.<locals>.get_run_class_method.<locals>.run_class_method of <syft.proxy.torch.devicePointer object at 0x133da33a0>>
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device', 'type'], 'index': 2} True torch.device
we are calling <class 'syft.ast.method.Method'> () {'path': ['torch', 'device', 'type'], 'index': 3} True syft.lib.python.String
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'String'], 'index': 4} True syft.lib.python.String
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device'], 'index': 2} True torch.device
executing a  <class 'syft.core.node.common.action.run_class_method_action.RunClassMethodAction'>
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device', 'type'], 'index': 2} False torch.device
we are calling <class 'syft.ast.method.Method'> () {'path': ['torch', 'device', 'type'], 'index': 3} False syft.lib.p

In [17]:
a = dtype.get()
print(a, type(a))

NameError: name 'dtype' is not defined

In [18]:
print(alice.store.peek)

{
  > <UID:d78bf8c7-d063-4add-ba79-46f85554ba69> <UID:🚴🙕> => 🗂️ (StorableObject) ('cpu' can_read: ['🚻', '🛺'])

  > <UID:b6763f66-dd2b-4a57-961f-18a47c650213> <UID:🙈🚇> => 🗂️ (StorableObject) (device(type='cpu') can_read: ['🚻', '🛺'])

  > <UID:3f8032c7-91d7-4041-a2e9-a0ea6baf69f6> <UID:🚍🛡> => 🗂️ (StorableObject) ('cpu' can_read: ['🚻', '🛺'])
}


In [19]:
import torch as local_torch

In [20]:
a = local_torch.Tensor([1])
print(a, type(a))

tensor([1.]) <class 'torch.Tensor'>


In [21]:
a.serialize()

id {
  value: "\214\365\334\320\242\025Fo\264\365\255\336\231=\220."
}
obj_type: "syft.lib.torch.uppercase_tensor.TorchTensorWrapper"
data {
  type_url: "type.googleapis.com/syft.lib.torch.TensorProto"
  value: "\n\023\n\001\001\022\007float32\262\001\004\000\000\200?"
}

In [22]:
model = Net()
print(alice.store.peek)

we are calling <class 'syft.ast.klass.Class'> () {} False models.MNIST
call ast with  models.MNIST True
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST'], 'index': 2} True models.MNIST
this is our return tensor <class 'syft.proxy.models.MNISTPointer'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST'], 'index': 2} False models.MNIST
we are here for <class 'syft.ast.klass.Class'> <class 'syft.lib.models.mnist.MNIST'>
{
  > <UID:d78bf8c7-d063-4add-ba79-46f85554ba69> <UID:🚴🙕> => 🗂️ (StorableObject) ('cpu' can_read: ['🚻', '🛺'])

  > <UID:b6763f66-dd2b-4a57-961f-18a47c650213> <UID:🙈🚇> => 🗂️ (StorableObject) (device(type='cpu') can_read: ['🚻', '🛺'])

  > <UID:3f8032c7-91d7-4041-a2e9-a0ea6baf69f6> <UID:🚍🛡> => 🗂️ (StorableObject) ('cpu' can_read: ['🚻', '🛺'])

  > <UID:3a70d51a-43fe-4d37-aba6-bb4073c38b1d> <UID:🚱🚂> => 🗂️ (StorableObject) (MNIST(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_siz

In [23]:
model = Net()
params = model.parameters()

we are calling <class 'syft.ast.klass.Class'> () {} False models.MNIST
call ast with  models.MNIST True
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST'], 'index': 2} True models.MNIST
this is our return tensor <class 'syft.proxy.models.MNISTPointer'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST'], 'index': 2} False models.MNIST
we are here for <class 'syft.ast.klass.Class'> <class 'syft.lib.models.mnist.MNIST'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST', 'parameters'], 'index': 2} True models.MNIST
we are calling <class 'syft.ast.method.Method'> () {'path': ['models', 'MNIST', 'parameters'], 'index': 3} True torch.nn.Parameter
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'nn', 'Parameter'], 'index': 3} True torch.nn.Parameter
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST'], 'index': 2} True models.MNIST
executing a  <class 'syft.core.node.c

In [24]:
# N X C, H, W
# make some data
list_ptr = make([64, 1, 28, 28])
x = torch.ones(list_ptr)
print(x, type(x))

we are calling <class 'syft.ast.function.Function'> (<syft.proxy.syft.lib.python.ListPointer object at 0x133d8bb80>,) {} False torch.Tensor
call ast with  torch.Tensor True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'Tensor'], 'index': 2} True torch.Tensor
this is our return tensor <class 'syft.proxy.torch.TensorPointer'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'List'], 'index': 4} True syft.lib.python.List
we are calling <class 'syft.ast.function.Function'> () {'path': ['torch', 'ones'], 'index': 2} False torch.Tensor
we are here for <class 'syft.ast.function.Function'> <built-in method ones of type object at 0x11ef885c0>
<syft.proxy.torch.TensorPointer object at 0x133e3da30> <class 'syft.proxy.torch.TensorPointer'>


In [25]:
model(x)

we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST', '__call__'], 'index': 2} True models.MNIST
we are calling <class 'syft.ast.method.Method'> () {'path': ['models', 'MNIST', '__call__'], 'index': 3} True models.MNIST
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST'], 'index': 2} True models.MNIST
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST'], 'index': 2} True models.MNIST
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'Tensor'], 'index': 2} True torch.Tensor
executing a  <class 'syft.core.node.common.action.run_class_method_action.RunClassMethodAction'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST', '__call__'], 'index': 2} False models.MNIST
we are calling <class 'syft.ast.method.Method'> () {'path': ['models', 'MNIST', '__call__'], 'index': 3} False models.MNIST
we are here for <class 'syft.ast.method.Method'> <function Module._call_impl at 0x12d

In [26]:
print(alice.store.peek)

{
  > <UID:d78bf8c7-d063-4add-ba79-46f85554ba69> <UID:🚴🙕> => 🗂️ (StorableObject) ('cpu' can_read: ['🚻', '🛺'])

  > <UID:b6763f66-dd2b-4a57-961f-18a47c650213> <UID:🙈🚇> => 🗂️ (StorableObject) (device(type='cpu') can_read: ['🚻', '🛺'])

  > <UID:3f8032c7-91d7-4041-a2e9-a0ea6baf69f6> <UID:🚍🛡> => 🗂️ (StorableObject) ('cpu' can_read: ['🚻', '🛺'])

  > <UID:3a70d51a-43fe-4d37-aba6-bb4073c38b1d> <UID:🚱🚂> => 🗂️ (StorableObject) (MNIST(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
))

  > <UID:5d45ed0c-98c9-499f-9308-025cf4afd216> <UID:🙿🛣> => 🗂️ (StorableObject) (MNIST(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout

In [27]:
params = model.parameters()

we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST', 'parameters'], 'index': 2} True models.MNIST
we are calling <class 'syft.ast.method.Method'> () {'path': ['models', 'MNIST', 'parameters'], 'index': 3} True torch.nn.Parameter
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'nn', 'Parameter'], 'index': 3} True torch.nn.Parameter
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST'], 'index': 2} True models.MNIST
executing a  <class 'syft.core.node.common.action.run_class_method_action.RunClassMethodAction'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['models', 'MNIST', 'parameters'], 'index': 2} False models.MNIST
we are calling <class 'syft.ast.method.Method'> () {'path': ['models', 'MNIST', 'parameters'], 'index': 3} False torch.nn.Parameter
we are here for <class 'syft.ast.method.Method'> <function Module.parameters at 0x12d5541f0>


In [28]:
# optimizer = optim.Adadelta(params, lr=make(1.0))
# TODO fix ValueError: optimizer got an empty parameter list
# might be incorrect return type for methods and properties

In [29]:
conv1 = nn.Conv2d(make(1), make(32), make(3), make(1))
print(conv1, type(conv1))

we are calling <class 'abc.SubclassableClass'> (<syft.proxy.syft.lib.python.IntPointer object at 0x133e3d610>, <syft.proxy.syft.lib.python.IntPointer object at 0x133e1cd60>, <syft.proxy.syft.lib.python.IntPointer object at 0x133e20f40>, <syft.proxy.syft.lib.python.IntPointer object at 0x133d5f3a0>) {} False torch.nn.Conv2d
call ast with  torch.nn.Conv2d True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'nn', 'Conv2d'], 'index': 3} True torch.nn.Conv2d
this is our return tensor <class 'syft.proxy.torch.nn.Conv2dPointer'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'Int'], 'index': 4} True syft.lib.python.Int
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'Int'], 'index': 4} True syft.lib.python.Int
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'Int'], 'index': 4} True syft.lib.python.Int
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syf

In [30]:
# test all the Functional methods

In [31]:
# N X C, H, W
# make some data
list_ptr2 = make([32, 1, 3, 3])
g = torch.ones(list_ptr)
print(g, type(g))

x = conv1(g)
x = F.relu(x)
x = F.relu(x)
x = F.max_pool2d(x, make(2))
x = torch.flatten(x, make(1))
x = F.relu(x)
output = F.log_softmax(x, dim=make(1))

we are calling <class 'syft.ast.function.Function'> (<syft.proxy.syft.lib.python.ListPointer object at 0x133d8bb80>,) {} False torch.Tensor
call ast with  torch.Tensor True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'Tensor'], 'index': 2} True torch.Tensor
this is our return tensor <class 'syft.proxy.torch.TensorPointer'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'List'], 'index': 4} True syft.lib.python.List
we are calling <class 'syft.ast.function.Function'> () {'path': ['torch', 'ones'], 'index': 2} False torch.Tensor
we are here for <class 'syft.ast.function.Function'> <built-in method ones of type object at 0x11ef885c0>
<syft.proxy.torch.TensorPointer object at 0x133e4f3a0> <class 'syft.proxy.torch.TensorPointer'>
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'nn', 'Conv2d', '__call__'], 'index': 3} True torch.nn.Conv2d
we are calling <class 'syft.ast.method.Method'> () {'path': ['torch', 'n

In [32]:
# get the output
print(output, type(output))

<syft.proxy.torch.TensorPointer object at 0x133e537f0> <class 'syft.proxy.torch.TensorPointer'>


In [33]:
result = output.get()
print(result)

/Users/madhavajay/dev/PySyft/src/syft/lib/torch/uppercase_tensor.py:55: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(self.value, "grad", None)


tensor([[-9.0029, -9.0029, -9.0029,  ..., -8.9366, -8.9366, -8.9366],
        [-9.0029, -9.0029, -9.0029,  ..., -8.9366, -8.9366, -8.9366],
        [-9.0029, -9.0029, -9.0029,  ..., -8.9366, -8.9366, -8.9366],
        ...,
        [-9.0029, -9.0029, -9.0029,  ..., -8.9366, -8.9366, -8.9366],
        [-9.0029, -9.0029, -9.0029,  ..., -8.9366, -8.9366, -8.9366],
        [-9.0029, -9.0029, -9.0029,  ..., -8.9366, -8.9366, -8.9366]])


In [34]:
# ORIGINAL MNIST NET
# class Net(nn.Module):
#     def __init__(self) -> None:
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, 1)
#         self.conv2 = nn.Conv2d(32, 64, 3, 1)
#         self.dropout1 = nn.Dropout2d(0.25)
#         self.dropout2 = nn.Dropout2d(0.5)
#         self.fc1 = nn.Linear(9216, 128)
#         self.fc2 = nn.Linear(128, 10)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.dropout1(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         output = F.log_softmax(x, dim=1)
#         return output

# define a Network using make
# class Net(nn.Module):
# # class Net:
#     def __init__(self) -> None:
#         self.conv1 = nn.Conv2d(make(1), make(32), make(3), make(1))
#         self.conv2 = nn.Conv2d(make(32), make(64), make(3), make(1))
#         self.dropout1 = nn.Dropout2d(make(0.25))
#         self.dropout2 = nn.Dropout2d(make(0.5))
#         self.fc1 = nn.Linear(make(9216), make(128))
#         self.fc2 = nn.Linear(make(128), make(10))

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, make(2))
#         x = self.dropout1(x)
#         x = torch.flatten(x, make(1))
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         output = F.log_softmax(x, dim=make(1))
#         return output

In [35]:
# ORIGINAL TRAIN
# def train(args, model, device, train_loader, optimizer, epoch):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss = F.nll_loss(output, target)
#         loss.backward()
#         optimizer.step()
#         if batch_idx % args.log_interval == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))
#             if args.dry_run:
#                 break

In [36]:
# ORIGINAL TEST
# def test(model, device, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         test_loss, correct, len(test_loader.dataset),
#         100. * correct / len(test_loader.dataset)))

In [37]:
# fix issue with 
transform_1 = torchvision.transforms.ToTensor()
transform_2 = torchvision.transforms.Normalize(make(0.1307), make(0.3081))

# transform_list = make([transform_1, transform_2])
# print(type(transform_list), transform_list)
# fix cant store pointer? what to do?
# transform = torchvision.transforms.Compose(make([transform_1, transform_2]))

we are calling <class 'abc.SubclassableClass'> () {} False torchvision.transforms.ToTensor
call ast with  torchvision.transforms.ToTensor True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torchvision', 'transforms', 'ToTensor'], 'index': 3} True torchvision.transforms.ToTensor
this is our return tensor <class 'syft.proxy.torchvision.transforms.ToTensorPointer'>
we are calling <class 'abc.SubclassableClass'> () {'path': ['torchvision', 'transforms', 'ToTensor'], 'index': 3} False torchvision.transforms.ToTensor
we are here for <class 'abc.SubclassableClass'> <class 'torchvision.transforms.transforms.ToTensor'>
we are calling <class 'abc.SubclassableClass'> (<syft.proxy.syft.lib.python.FloatPointer object at 0x13379ff40>, <syft.proxy.syft.lib.python.FloatPointer object at 0x133d6e3d0>) {} False torchvision.transforms.Normalize
call ast with  torchvision.transforms.Normalize True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torchvision', 'transforms', 'Norm

In [38]:
# utility to convert config to pointers
# todo replace with Dict
# need to fix TypeError: 'DictPointer' object is not subscriptable first
def wrap_args_dict(args: dict) -> dict:
    wrapped_args = {}
    for k, v in args.items():
        if sy.lib.python.primitive_factory.isprimitive(v):
            wrapped_args[k] = make(v)
    return wrapped_args

In [39]:
def main():
    # Training settings
    plain_args = {"batch_size":64, "test_batch_size":1000,
            "epochs":14, "lr":1.0, "gamma":0.7,
            "no_cuda":True, "dry_run":False, "seed":42,
            "log_interval":10, "save_model":False}

    args = wrap_args_dict(plain_args)

    use_cuda = not plain_args["no_cuda"] and torch.cuda.is_available()
    device_type_ptr = make("cuda" if use_cuda else "cpu")
    torch.manual_seed(args["seed"])

    device = torch.device(device_type_ptr)
    print("device pointer", device)

    kwargs = {'batch_size': args["batch_size"]}
    if use_cuda:
        kwargs.update(
            wrap_args_dict(
                {
                    'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True
                },
            )
        )
        
    transform_1 = torchvision.transforms.ToTensor()
    transform_2 = torchvision.transforms.Normalize(make(0.1307), make(0.3081))

    # fix cant store pointer? what to do?
    # transform = torchvision.transforms.Compose(make([transform_1, transform_2]))

    dataset1 = datasets.MNIST(make('../data'), train=make(True), download=make(True))
                       #transform=transform)
    dataset2 = datasets.MNIST(make('../data'), train=make(False))
#                        transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

    
    model = Net()
    model = model.to(device)
    # optimizer = optim.Adadelta(model.parameters(), lr=args["lr"])
    # TODO fix ValueError: optimizer got an empty parameter list
    # might be incorrect return type for methods and properties

    # scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
#     for epoch in range(1, args["epochs"].get() + 1):
#         train(args, model, device, train_loader, optimizer, epoch)
#         test(model, device, test_loader)
#         scheduler.step()

#     if args["save_model"]:
#         torch.save(model.state_dict(), "mnist_cnn.pt")

In [40]:
main()

we are calling <class 'syft.ast.function.Function'> (<syft.proxy.syft.lib.python.IntPointer object at 0x133e4f100>,) {} False torch.Generator
call ast with  torch.Generator True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'Generator'], 'index': 2} True torch.Generator
this is our return tensor <class 'syft.proxy.torch.GeneratorPointer'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'Int'], 'index': 4} True syft.lib.python.Int
we are calling <class 'syft.ast.function.Function'> () {'path': ['torch', 'manual_seed'], 'index': 2} False torch.Generator
we are here for <class 'syft.ast.function.Function'> <function manual_seed at 0x12d2e2280>
we are calling <class 'abc.SubclassableClass'> (<syft.proxy.syft.lib.python.StringPointer object at 0x133e5a820>,) {} False torch.device
call ast with  torch.device True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device'], 'index': 2} True torch.device
this is our 

In [41]:
# Training settings
plain_args = {"batch_size":64, "test_batch_size":1000,
        "epochs":14, "lr":1.0, "gamma":0.7,
        "no_cuda":True, "dry_run":False, "seed":42,
        "log_interval":10, "save_model":False}

args = wrap_args_dict(plain_args)

use_cuda = not plain_args["no_cuda"] and torch.cuda.is_available()
device_type_ptr = make("cuda" if use_cuda else "cpu")
torch.manual_seed(args["seed"])

device = torch.device(device_type_ptr)
print("device pointer", device)

kwargs = {'batch_size': args["batch_size"]}
if use_cuda:
    kwargs.update(
        wrap_args_dict(
            {
                'num_workers': 1,
                'pin_memory': True,
                'shuffle': True
            },
        )
    )

# transform_1 = torchvision.transforms.ToTensor()
# transform_2 = torchvision.transforms.Normalize(make(0.1307), make(0.3081))

# fix cant store pointer? what to do?
# transform = torchvision.transforms.Compose(make([transform_1, transform_2]))

dataset1 = datasets.MNIST(make('../data'), train=make(True), download=make(True))
                   #transform=transform)
dataset2 = datasets.MNIST(make('../data'), train=make(False))
#                        transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)


model = Net()
model = model.to(device)
# optimizer = optim.Adadelta(model.parameters(), lr=args["lr"

we are calling <class 'syft.ast.function.Function'> (<syft.proxy.syft.lib.python.IntPointer object at 0x133e4f100>,) {} False torch.Generator
call ast with  torch.Generator True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'Generator'], 'index': 2} True torch.Generator
this is our return tensor <class 'syft.proxy.torch.GeneratorPointer'>
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'Int'], 'index': 4} True syft.lib.python.Int
we are calling <class 'syft.ast.function.Function'> () {'path': ['torch', 'manual_seed'], 'index': 2} False torch.Generator
we are here for <class 'syft.ast.function.Function'> <function manual_seed at 0x12d2e2280>
we are calling <class 'abc.SubclassableClass'> (<syft.proxy.syft.lib.python.StringPointer object at 0x134cc50d0>,) {} False torch.device
call ast with  torch.device True
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'device'], 'index': 2} True torch.device
this is our 

In [ ]:
print(type(train_loader))

In [ ]:
a = train_loader.__iter__()
print(a)

In [46]:
b = a

TypeError: 'ListPointer' object is not subscriptable

In [42]:
for i in train_loader.__iter__():
    print("what did we get", type(i))
    break

we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'utils', 'data', 'DataLoader', '__iter__'], 'index': 4} True torch.utils.data.DataLoader
we are calling <class 'syft.ast.method.Method'> () {'path': ['torch', 'utils', 'data', 'DataLoader', '__iter__'], 'index': 5} True syft.lib.python.List
we are calling <class 'syft.ast.klass.Class'> () {'path': ['syft', 'lib', 'python', 'List'], 'index': 4} True syft.lib.python.List
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'utils', 'data', 'DataLoader'], 'index': 4} True torch.utils.data.DataLoader
executing a  <class 'syft.core.node.common.action.run_class_method_action.RunClassMethodAction'>
we are calling <class 'abc.SubclassableClass'> () {'path': ['torch', 'utils', 'data', 'DataLoader', '__iter__'], 'index': 4} False torch.utils.data.DataLoader
we are calling <class 'syft.ast.method.Method'> () {'path': ['torch', 'utils', 'data', 'DataLoader', '__iter__'], 'index': 5} False syft.lib.python.List
we a

TypeError: 'ListPointer' object is not iterable

In [ ]:
a = train_loader.__iter__()

In [ ]:
b = a.__getitem__(0)
print(b, type(b))

In [48]:
# if it works:
# -> we can get a single item each 
# sequential model
# 
# collect into a list
# get len, iterate through
# convert a pointer of a remote list to a list of pointers
# add optimiser
# add sequential
# scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

# if it doesnt work:
# -> 

In [47]:
a = nn.Sequential()

AttributeError: 'Module' object has no attribute 'Sequential'